In [2]:
import numpy as np
import random
import sys
import csv
import os
import re
from IPython.display import clear_output

from keras.layers import Dense, LSTM, Input, Embedding, Dropout
from keras.utils import np_utils
from keras.utils.data_utils import get_file
from keras.models import Model, load_model, Sequential
from keras.optimizers import Adam, RMSprop
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.callbacks import LambdaCallback

sys.path.append('C:\\Users\\Bonnie\\Desktop\\Jupyter\\project')
os.chdir('C:\\Users\\Bonnie\\Desktop\\Jupyter\\project')

Using TensorFlow backend.


In [3]:
print(os.getcwd())

C:\Users\Bonnie\Desktop\Jupyter\project


In [ ]:
#%% 원래 책에 있던 text_loader
# =============================================================================
# def text_loader(self, text_file = 'data\\aesop\\data.txt' ):
#         
#     with open(text_file, encoding='utf-8-sig') as f:
#         text = f.read()
#         
#     
#     self.start_story = '| '  * self.seq_length
#     
#     # 텍스트 정제    
#     text = text.lower()
#     text = self.start_story + text
#     text = text.replace('\n\n\n\n\n', self.start_story)
#     text = text.replace('\n',' ') 
#     
#     text = re.sub('  +', '. ', text).strip()
#     text = text.replace('..', '.')
#     
#     text = re.sub('([!"#$%&()*+,-./:;<=>?@[\]^_`{|}~])', r' \1 ', text)
#     text = re.sub('\s{2,}', ' ', text)
#     
#     self.text = text  
# =============================================================================

In [5]:
class text_LSTM:
    def __init__(self):
        self.seq_length = 20
        return
    
    
    def text_loader(self, text_file = 'C:\\Users\\Bonnie\\Desktop\\Jupyter\\project\\data.csv' ):
            
        with open(text_file, encoding='utf-8-sig') as f:
            text = f.read()
            
        self.start_story = '| '  * self.seq_length
        
        # 텍스트 정제    
        text = text.lower()
        text = self.start_story + text
        text = text.replace('\n\n\n\n\n', self.start_story) # 줄바꿈 5번 된건 끝나고 새로 시작하는걸로 처리
        
        text = re.sub('[\n]{2,}','\n', text) #
        
        text = text.replace('\n','\n ') # 랩에 맞춰서 새로 만듬
        
        #text = re.sub('  +', '. ', text).strip()
        #text = text.replace('..', '.')
        
        text = re.sub('([!"#$%&()*+,-./:;<=>?@[\]^_`{|}~])', r' \1 ', text) # 두번째 입력변수에 r' \1 ' 대신 ' \\1 ' 써줘도 똑같음 - r 써주는 방법이 더 안전할 수 있대
        #text = re.sub('\s{2,}', ' ', text)
        
        self.text = text
    
    def tokenize(self):
        # 토큰화
        self.tokenizer = Tokenizer(char_level = False, filters = '')         # 필터에 적혀있는 애는 토큰으로 안만든대
        self.tokenizer.fit_on_texts([self.text])                             # 여기서 리스트 하나 더 안씌우면 문자단위로 토큰화 한다.
        self.total_words = len(self.tokenizer.word_index) + 1                # 모든 단어종류 +1 // 1 더하는 이유는, 토크나이저.word_index가 1부터 인덱싱 해서, 뒤에 to_categorical로 원핫 인코딩할때 맞춰주기 위해서임
        self.token_list = self.tokenizer.texts_to_sequences([self.text])[0]  # text의 모든 단어들을 토큰으로 바꾼거    
    
    ### seq_length만큼 잘라서, 신경망에 학습시킬 수 있도록 각 seq - 다음단어 쌍을 만들어준다.
    
    def generate_sequences(self, step = 1):
        X = []
        y = []
        
        for i in range(0, len(self.token_list) - self.seq_length, step):
            X.append(self.token_list[i:i+self.seq_length])
            y.append(self.token_list[i + self.seq_length])
            
        y = np_utils.to_categorical(y, num_classes = self.total_words) 
        # 1차원짜리 y값을 원핫 인코딩으로 변환 (total_words 개수만큼)
        # to_categorical 얘는 그냥 0이면 1번쨰열을 1 나머지는 0, 55면 56번째 열을 1 나머지는 0 이런식으로
        # 숫자 값 그대로 원핫 인코딩 해주는 애
        
        self.X = np.array(X)
        self.y = np.array(y)
        self.num_seq = len(X)        
        
        print('시퀀스 개수:', self.num_seq, '\n')
    
    def build_network(self):
        
        n_units = 256
        embedding_size = 100
        
        # 여기서부터 신경망 시작
        
        text_in = Input(shape = (None,))
        x = Embedding(self.total_words, embedding_size)(text_in)
        x = LSTM(units = n_units)(x)    # output의 차원은 n_units 개
        x = Dropout(rate = 0.2)(x)
        text_out = Dense(units = self.total_words, activation = 'softmax')(x)   # 아웃풋은 단어 종류의 개수만큼 표현이 되어야 하니까
        
        self.model1 = Model(text_in, text_out)        
        
    def compile_network(self):
        
        optimizer = RMSprop(lr = 0.001)
        self.model1.compile(optimizer = optimizer, 
                            loss = 'categorical_crossentropy', 
                            metrics = ['accuracy'])
        
    def fit_network(self, epochs = 10, batch_size = 32):
        
        self.model1.fit(x = self.X, 
                        y = self.y, 
                        epochs = epochs, 
                        batch_size = batch_size, 
                        shuffle = True)    
        
    
    ### 100에폭 학습시켜놓은 모델 불러오기
        
    def load_network(self, model_file_path = 'saved_models\\text_model.h5'):
        print('load model...')
        self.model1 = load_model(model_file_path)
        print('model loaded : {0}'.format(model_file_path))

    def _sample_with_temp(self, preds, temperature = 1.0):
        # 확률 배열에서 인덱스 하나를 샘플링하는 함수
        # preds에는 원핫 카테고리별 확률값이 들어있는, 확률분포 리스트가 들어와야해.
        
        preds = np.array(preds).astype('float64')
        exp_preds = np.exp( np.log(preds) / temperature ) 
        preds = exp_preds / np.sum(exp_preds)  # 이게 확률분포
        probability = np.random.multinomial(1,preds,1) # 입력변수 각각 (주사위 몇번 던질래, 주사위 눈은몇개고 확률은 각각 어떻게 돼?, 이 결과값이 몇개 필요해?)
        
        return np.argmax(probability)
    
    def generate_text(self, seed_text, next_words, max_sequence_len=20 , temperature = 1):
        
        output_text = seed_text + ' '
        seed_text = self.start_story + seed_text
        
        for _ in range(next_words):
            token_list = self.tokenizer.texts_to_sequences([seed_text])[0]  # 글자들을 인덱스로 바꾼다. seed_text 겉에 리스트 씌워야 한다. 역으로 sequences_to_texts 할 때도 겉에 리스트 씌워야 한다.
            token_list = token_list[-max_sequence_len:]
            token_list = np.reshape(token_list, (1, max_sequence_len))  # model1.predict 에 넣어주려면 배치 형태로 넣어줘야 하니까 앞에 차원 하나 만들어줌
            #print(type(token_list)) # numpy array
            
            probs = self.model1.predict(token_list, verbose = 0)[0]  # 배치형태로 값이 들어갔으니까, 출력도 배치형태라서 [0] 으로 차원 하나 풀어준다.
            y_index = self._sample_with_temp(probs, temperature = temperature)
            
            output_word = self.tokenizer.index_word[y_index] if y_index > 0 else ''
            # 신경망을 통해 뽑은 숫자를 글자로 다시 바꿔준다
            # if절에서 y_index가 0보다 커야하는 조건 넣어준 이유는, tokenizer.index_word 딕셔너리는 key값 0이 없기 때문.
            
            if output_word == '|':
                break
            
            seed_text += output_word + ' '
            output_text += output_word + ' '
            
        return output_text
    
#### 단어 등장 확률 보고 싶어서 만들었다
    
    def _show_preds(self,preds):     
            
        preds = enumerate(preds)
        preds = sorted(preds, key = lambda x:x[1], reverse = True )[:10]
        
        for i in range(10):
            print('{0} : {1:10.2%} '.format(self.tokenizer.index_word[ preds[i][0] ], preds[i][1]))
            
    
    def generate_text2(self, seed_text, next_words=1, max_sequence_len=20 , temperature = 1):
        
        output_text = seed_text + ' '
        seed_text = self.start_story + seed_text
        
        for _ in range(next_words):
            token_list = self.tokenizer.texts_to_sequences([seed_text])[0]  # 글자들을 인덱스로 바꾼다. seed_text 겉에 리스트 씌워야 한다. 역으로 sequences_to_texts 할 때도 겉에 리스트 씌워야 한다.
            token_list = token_list[-max_sequence_len:]
            token_list = np.reshape(token_list, (1, max_sequence_len))
            #print(type(token_list)) # numpy array
            
            probs = self.model1.predict(token_list, verbose = 0)[0]
            self._show_preds(probs)
            

t1 = text_LSTM()
t1.text_loader(text_file = 'C:\\Users\\Bonnie\\Desktop\\Jupyter\\project\\data.csv')
t1.tokenize()
t1.generate_sequences()
#t1.load_network()

MemoryError: 

In [ ]:
tmp = 0
for i,j in t1.tokenizer.index_word.items():
    
    print(i,':',j)
    tmp += 1
    if tmp > 100 : break

t1.build_network()
t1.compile_network()
t1.fit_network(epochs = 5, batch_size = 32)

In [ ]:
print(t1.generate_text('All this dissing', 100, temperature = 0.1))
t1.generate_text2('All this dissing')

In [ ]:
aa = t1.tokenizer.texts_to_sequences( [t1.text])[0]
print(aa.count(2))

In [ ]:
def text_loader1(text_file = 'C:\\Users\\Bonnie\\Desktop\\Jupyter\\project\\data.csv' ):
        
    with open(text_file, encoding='utf-8-sig') as f:
        text = f.read()
        
    
    start_story = '| '  * 20
    
    # 텍스트 정제    
    text = text.lower()
    text = start_story + text
    text = text.replace('\n\n\n\n\n', start_story)
    
    text = re.sub('[\n]{2,}','\n\n', text) 
    
    #text = text.replace('\n' , '\n ') 
    #text = re.sub('\n+[^\s]' , r'\1 \n ',text) # 랩에 맞춰서 새로 만듬
    
    #text = re.sub('  +', '. ', text).strip()
    #text = text.replace('..', '.')
        
    text = re.sub('([!"#$%&()*+,-./:;<=>?@[\]^_`{|}~])', r' \1 ', text) # 두번째 입력변수에 r' \1 ' 대신 ' \\1 ' 써줘도 똑같음 - r 써주는 방법이 더 안전할 수 있대
    #text = re.sub('\s{5,}', ' ', text)
    
    return text

text = text_loader1()
print(text)

In [ ]:
seed_text = 'hello, this is'

output_text = seed_text
seed_text = '| '*20 + seed_text
print(seed_text)
token_list = t1.tokenizer.texts_to_sequences([seed_text])[0]  # 글자들을 인덱스로 바꾼다. seed_text 겉에 리스트 씌워야 한다. 역으로 sequences_to_texts 할 때도 겉에 리스트 씌워야 한다.
print(token_list)
token_list = token_list[-20:]
print(token_list)

In [ ]:
token_list = np.reshape(token_list, (1, 20))
print(token_list)

In [ ]:
# 테스트 해보는 코드들
mark = 613

a = t1.model2.predict(t1.X[mark:mark+1])

print(a.shape)

In [ ]:
print(type(X))

# print(X[10])
for i in X[mark]:
    print(tokenizer.index_word[i])
#print(tokenizer.index_word[2])
print(tokenizer.index_word[ np.argmax(a)])

In [ ]:
print( model2.evaluate(X[mark:mark+1],a))

In [ ]:
print(tokenizer.sequences_to_texts([[1,2,3,4,5]]) )

In [ ]:
np.random.multinomial(2, [1/6.]*6, size=1)

In [ ]:
for i in y[:3]:
    print(np.argmax(i))

In [ ]:
#print(token_list[:50])
aa = tokenizer.index_word
print(aa)

In [ ]:
y = [1,2,7,4,5]
y2 = np_utils.to_categorical(y,8)
print(y2)

In [6]:
path = 'data.csv'

with io.open(path, encoding = 'utf-8') as f :
    corpus = f.read().lower()
    
corpus = re.sub('\n', ' ', corpus)

corpus

NameError: name 'io' is not defined

In [10]:
corpus_length = len(corpus)
corpus_length

1057

In [11]:
words = sorted(list(set(corpus)))
total_words = len(words)
total_words

273

In [12]:
words_indices = dict((c,i) for i, c in enumerate(words))
indices_words = dict((i,c) for i, c in enumerate(words))

print(words_indices)
print(indices_words)

{' ': 0, '3': 1, '4': 2, 'a': 3, 'c': 4, 'd': 5, 'e': 6, 'f': 7, 'g': 8, 'h': 9, 'i': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'r': 17, 's': 18, 't': 19, 'u': 20, 'w': 21, 'x': 22, 'y': 23, '가': 24, '간': 25, '강': 26, '같': 27, '갚': 28, '개': 29, '거': 30, '건': 31, '것': 32, '게': 33, '겨': 34, '격': 35, '경': 36, '계': 37, '고': 38, '공': 39, '과': 40, '관': 41, '구': 42, '권': 43, '그': 44, '근': 45, '기': 46, '긴': 47, '까': 48, '깐': 49, '깽': 50, '꼭': 51, '꾼': 52, '끈': 53, '끝': 54, '나': 55, '난': 56, '날': 57, '남': 58, '내': 59, '냄': 60, '냐': 61, '너': 62, '넌': 63, '널': 64, '넘': 65, '네': 66, '녜': 67, '놈': 68, '누': 69, '느': 70, '는': 71, '니': 72, '다': 73, '닥': 74, '단': 75, '달': 76, '담': 77, '대': 78, '댈': 79, '더': 80, '덕': 81, '데': 82, '도': 83, '독': 84, '돈': 85, '동': 86, '돼': 87, '되': 88, '든': 89, '듣': 90, '들': 91, '등': 92, '딴': 93, '때': 94, '떤': 95, '똑': 96, '라': 97, '란': 98, '람': 99, '랑': 100, '래': 101, '랩': 102, '럭': 103, '럼': 104, '럽': 105, '레': 106, '려': 107, '력': 108, '로': 109, '론': 110,

In [19]:
max_length = 40
step = 3
sentences = []
next_chars = []

for i in range(0, len(corpus) - max_length, step) :
    sentences.append(corpus[i:i+max_length])
    next_chars.append(corpus[i+max_length])

print('nb sequences : ', len(sentences))

nb sequences :  339


In [22]:
x = np.zeros((len(sentences), max_length, len(words)), dtype = np.bool)
y = np.zeros((len(sentences), len(words)), dtype = np.bool)

for i, sentence in enumerate(sentences) :
    for j, word in enumerate(sentence) :
        x[i, j, words_indices[word]] = 1
    y[i, words_indices[next_chars[i]]] = 1

In [ ]:
model = Sequential()
model.add(LSTM(1024, input_shape = (max_length, len(words))))
model.add(Dense(len(words), activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy', optimizer = RMSprop(lr = 0.001))

In [ ]:
def sample(preds, temperature = 1.0) :
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.ramdon.multinomila(1, preds, 1)
    return np.argmax(probas)

def on_epoch_end(epoch, _) :
    start_index = random.randint(0, len(chrpus) - max_length -1)
    generated = ''
    sentence = corpus[start_index : start_index + max_length]
    generated += sentence
    sys.stdout.write(generated)
    
    for i in range(400) :
        x_pred[0, t, word_indices[word]] = 1
        next_index = sample(preds, 0.5)
        next_word = indices_word[next_index]
        
        generated += next_word
        sentence = sentence[1:] + next_word
        
        sys.stdout.write(next_word)
        sys,stdout.flush()
    print()
    
print_callback = LambdaCallback(on_epoch_end = on_epoch_end)

In [ ]:
model.fit(x, y, batch_size = 128, epochs = 60, callbacks = [print_callback])